In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [ ]:
!pip install geemap &> install.log

In [2]:
!pip install geopandas

In [4]:
import ee
import pandas as pd
import os
import math
import geopandas as gpd
import numpy as np
import os


In [5]:
try:
    ee.Initialize()
except:
    ee.Authenticate(auth_mode='notebook')
    ee.Initialize(project='potential-evap')

In [6]:
# TEST SHAPES LIST
shapefiles_list = []
finished_basins = []
path = "GAGES_smooth_shp"
finished_path = "Temp_TS"

for file in os.listdir(finished_path):
  finished_basins.append(file[-12:-4])


for file in os.listdir(path):
  if file.endswith('.shp'):
    file_path = os.path.join(path,file)
    basin_no = file_path[-12:-4] #string
    if basin_no not in finished_basins:
      shapefiles_list.append(file_path)

print(len(shapefiles_list))

1077


In [7]:
def extract_bits(image, start, end=None, new_name=None):
    """Function to convert qa bits to binary flag image

    args:
        image (ee.Image): qa image to extract bit from
        start (int): starting bit for flag
        end (int | None, optional): ending bit for flag, if None then will only use start bit. default = None
        new_name (str | None, optional): output name of resulting image, if None name will be {start}Bits. default = None

    returns:
        ee.Image: image with extract bits
    """

    newname = new_name if new_name is not None else f"{start}_bits"

    if (start == end) or (end is None):
        # perform a bit shift with bitwiseAnd
        return image.select([0], [newname]).bitwiseAnd(1 << start)
    else:
        # Compute the bits we need to extract.
        pattern = 0
        for i in range(start, end):
            pattern += int(math.pow(2, i))

        # Return a single band image of the extracted QA bits, giving the band
        # a new name.
        return image.select([0], [newname]).bitwiseAnd(pattern).rightShift(start)


In [8]:
bounds = None
def region_temp(image): # THESE MAPPED FUNCTIONS ONLY TAKE ONE ARGUEMENT (AN IMAGE)
    # reduction function
    temp = image.reduceRegion( #zonal statistics
        reducer = ee.Reducer.mean(),
        geometry = bounds,
        # geometry = Map.draw_features[0].geometry(),
        scale = 1000 # 1 km
    )

    # set the result as a metadata property in the image
    return image.set(temp)

In [9]:
def preprocess_day(image):
    qa_band = image.select("QC_Day")

    mask = extract_bits(qa_band, start=2, end=3).eq(0)

    return image.multiply(0.02).subtract(273.15).updateMask(mask).copyProperties(image,["system:time_start"])

In [10]:
def preprocess_night(image):
    qa_band = image.select("QC_Night")

    mask = extract_bits(qa_band, start=2, end=3).eq(0)

    return image.multiply(0.02).subtract(273.15).updateMask(mask).copyProperties(image,["system:time_start"])

In [11]:
modis_lst = ee.ImageCollection("MODIS/006/MYD11A2") #aqua
for i in range(len(shapefiles_list)):
# for i in range(1):
  shapefile = gpd.read_file(shapefiles_list[i])

  features = []
  for j in range(shapefile.shape[0]):
    geom = shapefile.iloc[j:j+1,:]
    jsonDict = eval(geom.to_json())
    geojsonDict = jsonDict['features'][0]
    features.append(ee.Feature(geojsonDict))

  fc = ee.FeatureCollection(features)
  bounds = fc.geometry().bounds()

  # Day
  lst_cel_day = modis_lst.map(preprocess_day).select("LST_Day_1km")
  lst_cel_region_day = lst_cel_day.map(region_temp).filter(ee.Filter.neq("LST_Day_1km",None))

  timeseries_day = lst_cel_region_day.aggregate_array("LST_Day_1km").getInfo()
  timestamp_day = lst_cel_region_day.aggregate_array("system:time_start").getInfo() #miliseconds since epoch

  dates_day = pd.to_datetime(np.array(timestamp_day)*1e6) #we have to scale the timestamp so it's in the proper units
  day_series = pd.Series(timeseries_day,index=dates_day,name="Region Day LST")

  # Night
  lst_cel_night = modis_lst.map(preprocess_night).select("LST_Night_1km")
  lst_cel_region_night = lst_cel_night.map(region_temp).filter(ee.Filter.neq("LST_Night_1km",None))

  timeseries_night = lst_cel_region_night.aggregate_array("LST_Night_1km").getInfo()
  timestamp_night = lst_cel_region_night.aggregate_array("system:time_start").getInfo() #miliseconds since epoch

  dates_night = pd.to_datetime(np.array(timestamp_night)*1e6) #we have to scale the timestamp so it's in the proper units
  night_series = pd.Series(timeseries_night,index=dates_night,name="Region Night LST")

  #-----------------------
  df = pd.concat([day_series, night_series], axis=1)
  df['Region Daily Average LST'] = (df['Region Day LST'] + df['Region Night LST'])/2

  # # df.plot(figsize=(15,11))
  # file_PathName = '/content/drive/MyDrive/IDeA Lab/Temperature/Basin_Temp_ts/' + 'basin_' + shapefiles_list[i][-11:-4] + '.csv'
  # print(shapefiles_list[i][-11:-4])
  # df.to_csv(path_or_buf=file_PathName,index=True)

  if len(df) > 0:
    df = df['Region Daily Average LST']

    file_PathName = 'Temp_TS/' + 'basin_' + shapefiles_list[i][-12:-4] + '.csv'
    print(i, '. YES \t', shapefiles_list[i][-12:-4])
    df.to_csv(path_or_buf=file_PathName,index=True)
    # print("exported!")
  else:
    df = pd.DataFrame(data={})
    file_PathName = 'Temp_TS/' + 'basin_' + shapefiles_list[i][-12:-4] + '.csv'
    df.to_csv(path_or_buf=file_PathName,index=True)
    print(i, '. NO \t', shapefiles_list[i][-12:-4])

0 . YES 	 06752260
1 . YES 	 11087020
2 . YES 	 07242350
3 . YES 	 05542000
4 . YES 	 11266500
5 . YES 	 07359002
6 . YES 	 02146750
7 . YES 	 08068390
8 . YES 	 11151300
9 . YES 	 07299540
10 . YES 	 05446500
11 . YES 	 09484580
12 . YES 	 06928300
13 . YES 	 15518000
14 . YES 	 09468500
15 . YES 	 10172800
16 . YES 	 06316400
17 . YES 	 02334430
18 . YES 	 02270500
19 . YES 	 06430850
20 . YES 	 03112000
21 . YES 	 02425000
22 . YES 	 09239500
23 . YES 	 02370500
24 . YES 	 09047700
25 . YES 	 01115670
26 . YES 	 01414000
27 . YES 	 14156500
28 . YES 	 06898400
29 . YES 	 05059600
30 . YES 	 01586210
31 . YES 	 08050800
32 . YES 	 01410500
33 . YES 	 14152000
34 . YES 	 10287400
35 . YES 	 01480675
36 . YES 	 12413125
37 . YES 	 02208450
38 . YES 	 03110830
39 . YES 	 01514000
40 . YES 	 14056500
41 . YES 	 09404343
42 . YES 	 03075070
43 . YES 	 04057510
44 . YES 	 02300018
45 . YES 	 01510500
46 . YES 	 14052000
47 . YES 	 08150800
48 . YES 	 03294550
49 . YES 	 06473000
50 . YES 	

#END

In [10]:
lst_cel_day = modis_lst.map(preprocess_day).select("LST_Day_1km")
lst_cel_region_day = lst_cel_day.map(region_temp).filter(ee.Filter.neq("LST_Day_1km",None))

SyntaxError: unexpected EOF while parsing (2357971886.py, line 56)

In [ ]:
lst_cel_night = modis_lst.map(preprocess_night).select("LST_Night_1km")
lst_cel_region_night = lst_cel_night.map(region_temp).filter(ee.Filter.neq("LST_Night_1km",None))

In [ ]:
timeseries_day = lst_cel_region_day.aggregate_array("LST_Day_1km").getInfo()
timestamp_day = lst_cel_region_day.aggregate_array("system:time_start").getInfo() #miliseconds since epoch

In [ ]:
dates_day = pd.to_datetime(np.array(timestamp_day)*1e6) #we have to scale the timestamp so it's in the proper units
day_series = pd.Series(timeseries_day,index=dates_day,name="Region Day LST")

In [ ]:
timeseries_night = lst_cel_region_night.aggregate_array("LST_Night_1km").getInfo()
timestamp_night = lst_cel_region_night.aggregate_array("system:time_start").getInfo() #miliseconds since epoch

In [ ]:
dates_night = pd.to_datetime(np.array(timestamp_night)*1e6) #we have to scale the timestamp so it's in the proper units
night_series = pd.Series(timeseries_night,index=dates_night,name="Region Night LST")

In [ ]:
df = pd.concat([day_series, night_series], axis=1)

In [ ]:
df['Region Daily Average LST'] = (df['Region Day LST'] + df['Region Night LST'])/2

In [ ]:
df_temp = pd.read_csv ('/content/drive/MyDrive/IDeA Lab/Temperature/provo_temp_gauge_noaa.csv', names=['Date','TAVG (Degrees Fahrenheit)', 'TMAX (Degrees Fahrenheit)',	'TMIN (Degrees Fahrenheit)',	'PRCP (Inches)',	'SNOW (Inches)',	'SNWD (Inches)'])